# How to use regioinvent

Note that running this entire notebook will take you around 2 to 3 hours if you choose the highest cutoff option.

To be able to use regioinvent, you will need:
- to install the brightway2 Python library (brightway2 and NOT brightway2.5), easier is to get it through activity-browser: https://github.com/LCA-ActivityBrowser/activity-browser
- a brightway project within which there is an ecoinvent database with either the version 3.9/3.9.1/3.10/3.10.1 cutoff
- to download the latest version of the trade database stored here: https://doi.org/10.5281/zenodo.11583814

In [1]:
import pandas as pd
import numpy as np
# import sys
# change the path here to wherever you stored the Regioinvent Python package 
# only needed if you've not installed through pip
#sys.path.append('C://Users/11max/PycharmProjects/Regioinvent/src/')
import regioinvent

12:09:38+0100 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [2]:
import bw2data
bw2data.projects.set_current("ecoinvent-3.10-cutoff")

In [3]:
bw2data.databases

Databases dictionary with 13 object(s):
	biosphere3
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ecoinvent-3.10.1-cutoff
	ei_cutoff_3.10.1_gcam_SSP2-Base_2050 2026-01-25
	ei_cutoff_3.10.1_image_SSP1-L_2025 2026-01-25
	ei_cutoff_3.10.1_remind_SSP1-NPi_2025 2026-01-25
	ei_cutoff_3.10.1_remind_SSP2-PkBudg650_2050 2026-02-04
	ei_cutoff_3.10.1_tiam-ucl_SSP2-RCP19_2050 2026-01-25
	h2_pem
	test1
	test2
	test3

In [3]:
#bw2io.ecoinvent.import_ecoinvent_release(
#    version="3.10.1",
#    system_model="cutoff",
#    username="Romain.Sacchi",
#    password="PSI!2023.x",
#    biosphere_name="biosphere3",
#)

The initialization of the Regioinvent class requires 3 arguments:
- the name of the brightway2 project where you stored ecoinvent and where regioinvent will be added
- the name you gave your ecoinvent database within your brightway2 project
- the version of the ecoinvent database ('3.9', '3.9.1', '3.10' or '3.10.1')

In [4]:
regio = regioinvent.Regioinvent(
    bw_project_name='ecoinvent-3.10-cutoff',
    ecoinvent_database_name='ecoinvent-3.10.1-cutoff',
    ecoinvent_version='3.10.1'
)

First step of regioinvent is to spatialize the original ecoinvent database. This entails two steps:
- Creating a new biosphere database for spatialized elementary flows (e.g., Ammonia, CA-QC)
- Spatializing the elementary flows used within the ecoinvent database, based on the location of the process itself

In [5]:
regio.spatialize_my_ecoinvent()

2026-02-20 12:10:29,452 - Regioinvent - INFO - Creating spatialized biosphere flows...
100%|████████████████████████████████| 110559/110559 [00:02<00:00, 38132.95it/s]

12:10:39+0100 [info     ] Vacuuming database            



2026-02-20 12:12:27,197 - Regioinvent - INFO - Extracting ecoinvent to wurst...


Getting activity data


100%|█████████████████████████████████| 23523/23523 [00:00<00:00, 297288.48it/s]


Adding exchange data to activities


100%|████████████████████████████████| 743409/743409 [00:23<00:00, 31459.32it/s]


Filling out exchange data


100%|██████████████████████████████████| 23523/23523 [00:01<00:00, 15406.56it/s]
2026-02-20 12:12:56,133 - Regioinvent - INFO - Spatializing ecoinvent...


This creates two databases in your brightway2 project:
- "_biosphere3_spatialized_flows_", which contains all the newly created spatialized elementary flows
- "the-name-of-your-ecoinvent-database regionalized", which is a copy of the original ecoinvent but using spatialized elementary flows
You thus end up with two ecoinvent versions: the original one and a spatialized one.

Because elementary flows are now spatialized, you will need a specific LCIA method to operate with any flow (spatialized or not). The following function imports such methods. Currently available: "IW v2.1", "EF v3.1", "ReCiPe 2016 v1.03 (H)". Can also import all of them in one go.

In [6]:
regio.import_fully_regionalized_impact_method()

2026-02-20 12:12:56,948 - Regioinvent - INFO - Importing all available fully regionalized lcia methods for ecoinvent3.10.
2026-02-20 12:14:56,540 - Regioinvent - INFO - EF v3.1 already present in Brightway project; skipping import.


If you want to go further in the regionalization, i.e., to create new national production processes and to rely on trade data to create regionalized consumption markets of ecoinvent, you can run the next function. There are 3 arguments:
- _trade_database_path_ which is the path where you stored the trade database you downloaded from Zenodo: https://doi.org/10.5281/zenodo.11583814
- _regioinvent_database_name_ which is simply the name that the created database will take
- _cutoff_ which is the cut-off (between 0 and 1) beyond which countries will be aggreated as RoW. For more info on what cutoff does, see section "Selection of countries for regionalization" of the Methodology.md file.

In [7]:
regio.regionalize_ecoinvent_with_trade(
    trade_database_path='/Users/romain/Documents/trade_data.db',
    cutoff=0.99
)

2026-02-20 12:16:01,186 - Regioinvent - INFO - Extracting and formatting trade data...
2026-02-20 12:16:10,506 - Regioinvent - INFO - Regionalizing main inputs of internationally-traded products of ecoinvent...
100%|███████████████████████████████████████| 1982/1982 [00:46<00:00, 42.99it/s]
2026-02-20 12:16:56,655 - Regioinvent - INFO - Regionalizing main inputs of non-internationally traded processes of ecoinvent...
100%|███████████████████████████████████████████| 67/67 [00:12<00:00,  5.27it/s]
2026-02-20 12:17:09,417 - Regioinvent - INFO - Creating consumption markets for internationally-traded products...
100%|███████████████████████████████████████| 1982/1982 [00:54<00:00, 36.66it/s]
2026-02-20 12:18:05,904 - Regioinvent - INFO - Link regioinvent processes to each other...
2026-02-20 12:18:24,809 - Regioinvent - INFO - Aggregate duplicates together...
2026-02-20 12:18:34,014 - Regioinvent - INFO - Regionalizing the elementary flows of the regioinvent database...
2026-02-20 12:18:3

In [8]:
regio.write_database()

2026-02-20 12:18:39,559 - Regioinvent - INFO - Write in-memory database to brightway...
2026-02-20 12:18:39,560 - Regioinvent - INFO - Normalizing in-memory datasets before write...
2026-02-20 12:18:59,972 - Regioinvent - INFO - Starting Brightway write...


12:19:01+0100 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████████████████████████████| 251593/251593 [07:52<00:00, 532.44it/s]


12:27:00+0100 [info     ] Vacuuming database            


This automatically wrote the regioinvent database in your brightway project. So you can now go on brightway2 or AB to perform your LCAs normally with regioinvent.

In [9]:
bw2data.databases

Databases dictionary with 15 object(s):
	biosphere3
	biosphere3_spatialized_flows
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ecoinvent-3.10.1-cutoff
	ecoinvent-3.10.1-cutoff - regionalized
	ei_cutoff_3.10.1_gcam_SSP2-Base_2050 2026-01-25
	ei_cutoff_3.10.1_image_SSP1-L_2025 2026-01-25
	ei_cutoff_3.10.1_remind_SSP1-NPi_2025 2026-01-25
	ei_cutoff_3.10.1_remind_SSP2-PkBudg650_2050 2026-02-04
	ei_cutoff_3.10.1_tiam-ucl_SSP2-RCP19_2050 2026-01-25
	h2_pem
	test1
	test2
	test3

In [10]:
len(bw2data.Database("ecoinvent-3.10.1-cutoff - regionalized"))

251593

In [11]:
len(bw2data.Database("Regioinvent"))

0

In [3]:
import bw2calc

In [4]:
db = bw2data.Database("Regioinvent")
results = []
for _ in range(6):
    print("yes")
    ds = db.random()
    lca = bw2calc.LCA({ds:1}, method=('IPCC 2021', 'climate change: fossil, including SLCFs', 'global warming potential (GWP100)'))
    lca.lci()
    lca.lcia()
    results.append(
        [
            ds["name"],
            ds["reference product"],
            ds["location"],
            lca.score
        ]
    )

import pandas as pd

pd.DataFrame(results)

yes


/opt/homebrew/Caskroom/miniforge/base/envs/regioinvent/lib/python3.12/site-packages/scikits/umfpack/umfpack.py:737: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 7.59e+13)
  warnings.warn(msg, UmfpackWarning)


yes
yes
yes
yes
yes


,0,1,2,3
0,"consumption market for ventilation system, dec...","ventilation system, decentralized, 6 x 120 m3/...",IN,11111.221433
1,consumption market for gold,gold,ZW,49751.993915
2,consumption market for ventilation of dwelling...,"ventilation of dwellings, central, 1 x 720 m3/h",TZ,3.029299
3,consumption market for photovoltaic slanted-ro...,"photovoltaic slanted-roof installation, 3kWp, ...",FR,8066.131281
4,consumption market for isoproturon,isoproturon,IE,6.717141
5,consumption market for anthranilic acid,anthranilic acid,CZ,5.532118


In [5]:
results

[['consumption market for ventilation system, decentralized, 6 x 120 m3/h, polyethylene ducts, with earth tube heat exchanger',
  'ventilation system, decentralized, 6 x 120 m3/h, polyethylene ducts, with earth tube heat exchanger',
  'IN',
  11111.221433415523],
 ['consumption market for gold', 'gold', 'ZW', 49751.99391503114],
 ['consumption market for ventilation of dwellings, central, 1 x 720 m3/h',
  'ventilation of dwellings, central, 1 x 720 m3/h',
  'TZ',
  3.0292988252818187],
 ['consumption market for photovoltaic slanted-roof installation, 3kWp, single-Si, laminated, integrated, on roof',
  'photovoltaic slanted-roof installation, 3kWp, single-Si, laminated, integrated, on roof',
  'FR',
  8066.13128068786],
 ['consumption market for isoproturon',
  'isoproturon',
  'IE',
  6.717140836099581],
 ['consumption market for anthranilic acid',
  'anthranilic acid',
  'CZ',
  5.532117529197189]]